# Build ACH Model - Dev Time
---


In [1]:
## first merge modeling_df and test_df with the newly created features

import json, os, sys
sys.path.insert(1, "../..")
from src.utils import remove_prefixs, _to_parquet
import pandas as pd
import numpy as np
import pickle as pkl
from rdsutils.aws import download_s3 #, python_object_to_s3
from rdsutils.lightgbm_helpers import train_lgb_baseline_grouped
import rdsutils.plot as rdsplot

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
%load_ext autoreload
%autoreload 2

In [2]:
with open("../../config.json", "r") as f:
    config = json.load(f)
    

path = config["data"]["labeled"]["labeled"]
print(path)
df = pd.read_parquet(os.path.join("../../data", path))

labeled/labeled_1623399221/labeled_1623399221.parquet


In [3]:
modeling_df = df[df["transaction_datetime"].between(pd.to_datetime('2020-05-01'),
                                                    pd.to_datetime('2020-07-31'))]
test_df = df[df["transaction_datetime"].between(pd.to_datetime('2020-08-01'),
                                                pd.to_datetime('2021-05-31'))]

In [4]:
modeling_df.columns[modeling_df.columns.str.contains("is_return")]

Index(['trnx_is_return', 'is_return', 'is_return_ach', 'is_return_mcd',
       'is_return_other', 'is_returned'],
      dtype='object')

### Data Statistics

In [5]:
modeling_df.is_returned.value_counts(), modeling_df.is_returned.value_counts(normalize=True)

(False    108825
 True       3774
 Name: is_returned, dtype: int64,
 False    0.966483
 True     0.033517
 Name: is_returned, dtype: float64)

In [6]:
test_df.is_returned.value_counts(), test_df.is_returned.value_counts(normalize=True)

(False    556896
 True      25596
 Name: is_returned, dtype: int64,
 False    0.956058
 True     0.043942
 Name: is_returned, dtype: float64)

In [7]:
modeling_df.columns = remove_prefixs(modeling_df.columns)
test_df.columns = remove_prefixs(test_df.columns)

In [8]:
1/modeling_df.is_returned.value_counts(normalize=True).loc[True]-1

28.835453100158983

In [9]:
modeling_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [10]:
_to_parquet(modeling_df.reset_index(), 's3://sofi-data-science/jxu/money-risk-models/deposit-risk-v2-ach/data/2021-05-19/modeling_df.parquet')
_to_parquet(test_df.reset_index(), 's3://sofi-data-science/jxu/money-risk-models/deposit-risk-v2-ach/data/2021-05-19/test_df.parquet')